## Extract and parse meaningful evalues for those with ignored, null, or noisy evalues

Ignored evalue example:

    fname                             00000-1828-homework1-1.ipynb
    ename                                                NameError
    evalue                                                 ignored
    traceback    ['---------------------------------------------------------------------------', 'NameError                                 Traceback (most recent call last)', '<ipython-input-8-287bf352fb78> in <module>()\n      3 c1= nd.norm(nd.dot(A, B))\n      4 c= mx.ndarray.power(c1,2)\n----> 5 tic= time.time()\n      6 print(time.time() - tic)\n      7 \n', "NameError: name 'time' is not defined"]


Noisy evalue example:

When clustering valueerror for GitHub error dataset, found that many noise points whose evalues are very long, for example:

    "in user code:

        R:\Work\Anacond\lib\site-packages\keras\engine\training.py:830 train_function  *
            return step_function(self, iterator)
        R:\Work\Anacond\lib\site-packages\keras\engine\training.py:813 run_step  *
            outputs = model.train_step(data)
        R:\Work\Anacond\lib\site-packages\keras\engine\training.py:771 train_step  *
            loss = self.compiled_loss(
        R:\Work\Anacond\lib\site-packages\keras\engine\compile_utils.py:201 __call__  *
            loss_value = loss_obj(y_t, y_p, sample_weight=sw)
        R:\Work\Anacond\lib\site-packages\keras\losses.py:142 __call__  *
            losses = call_fn(y_true, y_pred)
        R:\Work\Anacond\lib\site-packages\keras\losses.py:246 call  *
            return ag_fn(y_true, y_pred, **self._fn_kwargs)
        R:\Work\Anacond\lib\site-packages\tensorflow\python\util\dispatch.py:206 wrapper  **
            return target(*args, **kwargs)
        R:\Work\Anacond\lib\site-packages\keras\losses.py:1742 binary_crossentropy
            y_true, y_pred, from_logits=from_logits), axis=-1)
        R:\Work\Anacond\lib\site-packages\tensorflow\python\util\dispatch.py:206 wrapper
            return target(*args, **kwargs)
        R:\Work\Anacond\lib\site-packages\keras\backend.py:4988 binary_crossentropy
            return tf.compat.v1.nn.sigmoid_cross_entropy_with_logits(labels=target, logits=output)
        R:\Work\Anacond\lib\site-packages\tensorflow\python\util\dispatch.py:206 wrapper
            return target(*args, **kwargs)
        R:\Work\Anacond\lib\site-packages\tensorflow\python\ops\nn_impl.py:133 sigmoid_cross_entropy_with_logits
            (logits.get_shape(), labels.get_shape()))

        ValueError: logits and labels must have the same shape ((None, 150, 99) vs (None, 99))
    "
where only the last sentence should be there.we want to denoise those evalues in a domain-specific way.

### 1. nb errors in Kaggle

In [2]:
import pandas as pd
import util
import config

path_err = config.path_kaggle_error_process.joinpath('nberror_k_eid_p.xlsx') #nberror_k #original error data
df_err = pd.read_excel(path_err)
# df_err_ori = pd.read_excel(config.path_kaggle_error_process.joinpath("nberror_k_eid.xlsx"))

In [2]:
### Extract meaningful and short evalues from traceback for those with ignored evalues

In [4]:
# before
df_err_ignored = df_err[df_err['evalue']=='ignored']
df_err_ignored.count()

fname             175
eid               175
ename             175
evalue            175
traceback         175
ename_mapped      175
imports           175
lib_alias         175
python_version    175
if_ast            124
is_MLnb           175
is_relevant       175
dtype: int64

In [5]:
for index, row in df_err[(df_err['evalue']=='ignored')|df_err.evalue.isnull()].iterrows():
    res = util.get_evalue_ignored_from_traceback(row)
#     print(index, res)
    if res != None:
        df_err.loc[index, "evalue"] = res
        
# after
df_err[df_err['evalue']=='ignored'].count()

exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception 

fname             2
eid               2
ename             2
evalue            2
traceback         2
ename_mapped      2
imports           2
lib_alias         2
python_version    2
if_ast            0
is_MLnb           2
is_relevant       2
dtype: int64

In [9]:
# observe evalue == nan, there is actually no evalue
for index, row in df_err[df_err.evalue==''].iterrows():
    print(index, row.ename)

2 keyboardinterrupt
12 keyboardinterrupt
25 keyboardinterrupt
64 keyboardinterrupt
70 keyboardinterrupt
71 keyboardinterrupt
74 keyboardinterrupt
90 keyboardinterrupt
94 keyboardinterrupt
96 keyboardinterrupt
114 keyboardinterrupt
115 keyboardinterrupt
131 keyboardinterrupt
136 keyboardinterrupt
144 keyboardinterrupt
147 keyboardinterrupt
152 keyboardinterrupt
155 keyboardinterrupt
181 keyboardinterrupt
183 keyboardinterrupt
188 keyboardinterrupt
189 keyboardinterrupt
190 assertionerror
195 keyboardinterrupt
196 keyboardinterrupt
198 keyboardinterrupt
199 keyboardinterrupt
203 keyboardinterrupt
205 keyboardinterrupt
206 keyboardinterrupt
208 keyboardinterrupt
209 keyboardinterrupt
218 keyboardinterrupt
254 keyboardinterrupt
259 keyboardinterrupt
268 keyboardinterrupt
276 keyboardinterrupt
277 keyboardinterrupt
278 keyboardinterrupt
279 keyboardinterrupt
288 keyboardinterrupt
296 keyboardinterrupt
312 keyboardinterrupt
324 keyboardinterrupt
328 keyboardinterrupt
330 keyboardinterrupt
33

In [11]:
util.print_traceback(df_err.loc[1659, "traceback"])

---------------------------------------------------------------------------
AssertionError                            Traceback (most recent call last)
/tmp/ipykernel_26/1276441181.py in <module>
----> 1 assert translate(".") == "."
      2 assert translate("I walk around Paris") == "je marcher autour Paris"

AssertionError: 


In [12]:
## for those noisy evalues
i = 0
for index, row in df_err.iterrows():
    res = util.process_noisy_evalues(row)
    if res != None:
        df_err.loc[index, "evalue"] = res
        i+=1
        #print(index, res)
print(i)

407


In [13]:
df_err.to_excel(path_err, index=False, engine='xlsxwriter')

### 2. nb errors in Github

In [14]:
import pandas as pd
import util
import config

df2_err = pd.read_excel(config.path_github_error_process.joinpath("nberror_g_all_eid_p.xlsx"))

In [15]:
# for ignored and null values
df2_err_ignored = df2_err[df2_err['evalue']=='ignored']
df2_err_ignored.count()

fname             17355
eid               17355
ename             17355
evalue            17355
traceback         17355
ename_mapped      17355
imports           17354
lib_alias         17354
python_version    17355
if_ast            17222
is_MLnb           17355
is_relevant       17355
dtype: int64

In [16]:
for index, row in df2_err[(df2_err['evalue']=='ignored')|df2_err.evalue.isnull()].iterrows():
    res = util.get_evalue_ignored_from_traceback(row)
#     print(index, res)
    if res != None:
        df2_err.loc[index, "evalue"] = res
        
# after
df2_err[df2_err['evalue']=='ignored'].count()

exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception when listing traceback
exception 

fname             98
eid               98
ename             98
evalue            98
traceback         98
ename_mapped      98
imports           98
lib_alias         98
python_version    98
if_ast            97
is_MLnb           98
is_relevant       98
dtype: int64

In [17]:
# observe evalue == nan, there is actually no evalue
for index, row in df2_err[df2_err.evalue==''].iterrows():
    print(index, row.ename)

0 keyboardinterrupt
7 keyboardinterrupt
9 keyboardinterrupt
31 keyboardinterrupt
32 keyboardinterrupt
41 keyboardinterrupt
49 keyboardinterrupt
55 keyboardinterrupt
56 keyboardinterrupt
65 memoryerror
72 keyboardinterrupt
77 keyboardinterrupt
98 assertionerror
110 keyboardinterrupt
119 keyboardinterrupt
120 keyboardinterrupt
125 keyboardinterrupt
163 keyboardinterrupt
173 keyboardinterrupt
177 keyboardinterrupt
178 assertionerror
185 keyboardinterrupt
194 keyboardinterrupt
224 keyboardinterrupt
226 keyboardinterrupt
227 keyboardinterrupt
245 keyboardinterrupt
271 keyboardinterrupt
272 keyboardinterrupt
277 keyboardinterrupt
288 keyboardinterrupt
290 keyboardinterrupt
292 keyboardinterrupt
296 keyboardinterrupt
298 keyboardinterrupt
308 keyboardinterrupt
311 keyboardinterrupt
312 keyboardinterrupt
317 keyboardinterrupt
319 keyboardinterrupt
334 keyboardinterrupt
335 keyboardinterrupt
338 keyboardinterrupt
352 keyboardinterrupt
363 keyboardinterrupt
365 keyboardinterrupt
370 notimplement

53342 keyboardinterrupt
53343 keyboardinterrupt
53347 keyboardinterrupt
53383 keyboardinterrupt
53389 keyboardinterrupt
53398 keyboardinterrupt
53399 keyboardinterrupt
53415 keyboardinterrupt
53439 keyboardinterrupt
53444 keyboardinterrupt
53446 badsegmentationparameters
53452 keyboardinterrupt
53453 keyboardinterrupt
53459 keyboardinterrupt
53476 keyboardinterrupt
53481 keyboardinterrupt
53484 keyboardinterrupt
53503 keyboardinterrupt
53519 assertionerror
53520 assertionerror
53524 assertionerror
53525 assertionerror
53532 bdbquit
53533 keyboardinterrupt
53534 keyboardinterrupt
53550 memoryerror
53551 memoryerror
53566 keyboardinterrupt
53567 keyboardinterrupt
53569 keyboardinterrupt
53575 keyboardinterrupt
53583 keyboardinterrupt
53587 keyboardinterrupt
53601 assertionerror
53602 keyboardinterrupt
53603 keyboardinterrupt
53608 stopiteration
53613 keyboardinterrupt
53624 assertionerror
53626 keyboardinterrupt
53631 keyboardinterrupt
53632 keyboardinterrupt
53633 keyboardinterrupt
5364

105976 keyboardinterrupt
105985 keyboardinterrupt
105993 keyboardinterrupt
105998 keyboardinterrupt
105999 memoryerror
106006 keyboardinterrupt
106017 keyboardinterrupt
106019 assertionerror
106023 keyboardinterrupt
106026 keyboardinterrupt
106039 keyboardinterrupt
106041 keyboardinterrupt
106045 keyboardinterrupt
106068 stopiteration
106069 stopiteration
106070 stopiteration
106071 stopiteration
106072 typeerror
106073 stopiteration
106081 keyboardinterrupt
106093 keyboardinterrupt
106113 keyboardinterrupt
106130 keyboardinterrupt
106132 keyboardinterrupt
106143 keyboardinterrupt
106145 keyboardinterrupt
106146 keyboardinterrupt
106150 keyboardinterrupt
106161 keyboardinterrupt
106169 keyboardinterrupt
106183 keyboardinterrupt
106190 keyboardinterrupt
106209 keyboardinterrupt
106226 assertionerror
106227 assertionerror
106234 keyboardinterrupt
106249 reactornotrestartable
106279 keyboardinterrupt
106280 keyboardinterrupt
106326 keyboardinterrupt
106349 keyboardinterrupt
106374 keyboar

147453 keyboardinterrupt
147455 keyboardinterrupt
147457 keyboardinterrupt
147467 keyboardinterrupt
147470 keyboardinterrupt
147490 keyboardinterrupt
147494 keyboardinterrupt
147515 keyboardinterrupt
147522 keyboardinterrupt
147527 keyboardinterrupt
147528 keyboardinterrupt
147529 keyboardinterrupt
147537 keyboardinterrupt
147538 keyboardinterrupt
147539 keyboardinterrupt
147546 stopiteration
147551 keyboardinterrupt
147556 keyboardinterrupt
147575 keyboardinterrupt
147576 keyboardinterrupt
147577 keyboardinterrupt
147584 ownmath
147585 keyboardinterrupt
147588 keyboardinterrupt
147591 keyboardinterrupt
147595 keyboardinterrupt
147629 keyboardinterrupt
147630 keyboardinterrupt
147631 keyboardinterrupt
147632 keyboardinterrupt
147639 keyboardinterrupt
147640 assertionerror
147649 keyboardinterrupt
147671 keyboardinterrupt
147692 keyboardinterrupt
147702 keyboardinterrupt
147705 ipynb_exit
147715 keyboardinterrupt
147743 keyboardinterrupt
147750 keyboardinterrupt
147752 bdbquit
147762 ke

In [18]:
util.print_traceback(df2_err.loc[601, "traceback"])

---------------------------------------------------------------------------
Mismatch                                  Traceback (most recent call last)
<ipython-input-29-a2e59b9f91af> in <module>
----> 1 factorial('hello')

<nofile> in factorial(value)

Mismatch: 


In [19]:
## for those noisy evalues
i = 0
for index, row in df2_err.iterrows():
    res = util.process_noisy_evalues(row)
    if res != None:
        df2_err.loc[index, "evalue"] = res
        i+=1
        #print(index, res)
print(i)

5669


In [20]:
df2_err.to_excel(config.path_github_error_process.joinpath("nberror_g_all_eid_p.xlsx"), index=False, engine='xlsxwriter')